Step one is to make a class that sort of spits out new classes that are subclasses of the given class and the new class

In [2]:
class KType:
    registry = {}
    
    @classmethod
    def __class_getitem__(cls, x):   
        t = cls.registry.get(x)
        if t is not None:
            return t
        cls.registry[x] = type(f'ktype_{x.__name__}', (x, cls), {})    
        return cls.registry[x]
    
    @classmethod
    def __subclasscheck__(cls, o):
        return o in cls.registry

# Check that shit out
assert issubclass(KType[int], KType)

Step 2 is to make a class that checks for type annotations of this type and save them for later

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
   
    def __repr__(self) -> None:
        return f'{self.__class__.__name__}(x={self.x})'
     
    @classmethod
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.ktypes = {
            name: anno
            for name, anno in cls.__init__.__annotations__.items()
            if name != 'return' and issubclass(anno, KType)
        }

Step 3 is to go wild and make classes like this

In [3]:
          
class B(A):
    def __init__(self, x: int, y: KType[str]) -> None:
        super().__init__(x)
        self.y = y
        
    def __repr__(self):
        return f'{self.__class__.__name__}(x={self.x}, y={self.y})'
        
        
class C(B):
    def __init__(self, x: int, y: KType[str], z: KType[int]) -> None:
        super().__init__(x, y)
        self.z = z
        
    def __repr__(self):
        return f'{self.__class__.__name__}(x={self.x}, y={self.y}, z={self.z})'
    
    
B.ktypes, C.ktypes

({'y': __main__.ktype_str}, {'y': __main__.ktype_str, 'z': __main__.ktype_int})